In [1]:
import time
import os
import logging
import requests
import re
from dataclasses import dataclass
from random import randint
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent

In [2]:

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

def return_id_ele(name, soup):
    return soup.find('h5', text=name).next_element.next_element.next_element.next_element.next_element['id']


In [22]:
@dataclass
class Person:
    """Class for keeping track of an item in inventory."""
    first_name: str
    last_name: str
    third_name: str
    phone: str
    id_service_type: str  #  'نوع المعاملة'
    name_cir: str # اسم دائرة الاحوال
    meal: str #  الوجبة
    familly_number: str # 'عدد افراد الاسرة'
    confirmation: int



In [18]:
def open_driver():
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    logging.info(userAgent)
    options.add_argument(f'user-agent={userAgent}')
    #opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
    driver = webdriver.Firefox(firefox_options=options)
    return driver

In [66]:
def get_data_first_page(person: Person, driver):
    waiting = 90
    # Enter name1 --- > phone
    wait = WebDriverWait(driver, 20)
    first_name = WebDriverWait(driver,waiting).until(EC.visibility_of_element_located((By.XPATH,'//input[@name="FirstName"]')))
    first_name.send_keys(person.first_name)
    last_name = driver.find_element_by_xpath('//input[@name="SecondName"]').send_keys(person.last_name)
    third_name = driver.find_element_by_xpath('//input[@name="ThirdName"]').send_keys(person.third_name)
    phone = driver.find_element_by_xpath('//input[@name="Phone"]').send_keys(person.phone)

    # Fresh data
    r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(r, "html.parser")
    
    # Set servie type 'نوع المعاملة'
    id_service_type = return_id_ele('نوع المعاملة', soup)
    time.sleep(2)
    wait.until(EC.element_to_be_clickable((By.ID, f'{id_service_type}'))).click()
    print("id: ", person.id_service_type)
    wait.until(EC.element_to_be_clickable((By.XPATH, f'//li[@aria-label="{person.id_service_type}"]'))).click()
    # driver.find_element_by_id(f'{id_service_type}').click()
    # driver.find_element_by_xpath(f'//li[@aria-label="{person.id_service_type}"]').click()
    
    # Set Name circonsiption 'اسم دائرة الاحوال'
    name_cir = return_id_ele('اسم دائرة الاحوال', soup)
    # driver.find_element_by_id(f'{name_cir}').click()
    # driver.find_element_by_xpath(f'//li[@aria-label="{person.name_cir}"]').click()
    wait.until(EC.element_to_be_clickable((By.ID, f'{name_cir}'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, f'//li[@aria-label="{person.name_cir}"]'))).click()
    
    # Set Meal الوجبة
    name_toto = return_id_ele('الوجبة', soup)
    # driver.find_element_by_id(f'{name_toto}').click()
    # driver.find_element_by_xpath(f'//input[@id="search-{name_toto}"]').click()
    # driver.find_element_by_xpath(f'//li[@aria-label="{person.meal}"]').click()
    wait.until(EC.element_to_be_clickable((By.ID, f'{name_toto}'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, f'//input[@id="search-{name_toto}"]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, f'//li[@aria-label="{person.meal}"]'))).click()
    
    # Set Family number 'عدد افراد الاسرة'
    familly_number = return_id_ele('عدد افراد الاسرة', soup)
    # driver.find_element_by_id(f'{familly_number}').click()
    # driver.find_element_by_xpath(f'//input[@id="search-{familly_number}"]').click()
    # driver.find_element_by_xpath(f'//li[@aria-label="{person.familly_number}"]').click()
    wait.until(EC.element_to_be_clickable((By.ID, f'{familly_number}'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, f'//input[@id="search-{familly_number}"]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, f'//li[@aria-label="{person.familly_number}"]'))).click()
    
    # Click next
    enter_button = WebDriverWait(driver,waiting).until(EC.visibility_of_element_located((By.XPATH,'//span[text()="تسجيل الدخول"]')))
    enter_button.click()

def get_confirmation_page(person: Person, driver):
    waiting = 90
    wait = WebDriverWait(driver, 20)
    confirme_input = WebDriverWait(driver,waiting).until(EC.visibility_of_element_located((By.XPATH,'//input[@name="ConfirmCode"]')))
    confirme_input.send_keys(person.confirmation)
    time.sleep(2)
    # register_btn = driver.find_element_by_class_name('rz-button-text')
    register_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'rz-button-text')))
    register_btn.click()
    time.sleep(4)
    # confirmation_btn = driver.find_element_by_class_name('btn-success')
    confirmation_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn-success')))
    confirmation_btn.click()
    click_for_confirmation = 'اضغط هنا لإكمال عملية الحجز'
    return driver.find_element_by_xpath(f'//span[text()="{click_for_confirmation}"]').text


In [ ]:
url = 'https://reg.nid-moi.gov.iq/'

In [30]:
WORNG_MESSAGE = 'رقم الهاتف تم الحجز به مسبقاً يرجى الانتظار 30 يوم للحجز مرة اخرى'
SEVICE_TYPE_LIST = [
    '>الواقعات الحياتية' ,
    '>تسجيل لأول مرة',
]

NAME_CIRCONSIPTION_LIST = [
    '>دائرة احوال الرصافة' ,
    '>دائرة احوال بغداد الجديدة' ,
    '>دائرة احوال الكرادة الشرقية' ,
    '>دائرة احوال الراشدية' ,
    '>دائرة احوال الموصل الايمن' ,
]

MEAL_LIST = [
    '>صباحي'
]

NUMBER_FAMILY_LIST =[
    '>1',
    '>2',
    '>3',
    '>4',
    '>4',
    '>5',
    '>6',
    '>7',
    '>8',
    '>9',
]

In [17]:
def open_driver():
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    logging.info(userAgent)
    options.add_argument(f'user-agent={userAgent}')
    #opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
    driver = webdriver.Firefox(firefox_options=options)
    return driver

In [41]:
person1 = Person(
    'علي',
    'هاشم',
    'صالح',
    '07806840569',
    '>تسجيل لأول مرة',
    NAME_CIRCONSIPTION_LIST[4],
    MEAL_LIST[0],
    NUMBER_FAMILY_LIST[5],
    364536,
)
person2 = Person(
    'عبدالله',
    'رامي',
    'موفق',
    '07711037549',
    SEVICE_TYPE_LIST[1],
    NAME_CIRCONSIPTION_LIST[4],
    MEAL_LIST[0],
    NUMBER_FAMILY_LIST[3],
    951864,
)

In [31]:
SEVICE_TYPE_LIST[1]

'>تسجيل لأول مرة'

In [67]:
driver = open_driver()
driver.get('https://reg.nid-moi.gov.iq/')
get_data_first_page(person1, driver)
logging.info('Go Reservation Page.')
time.sleep(1)
RESERVATION = get_confirmation_page(person1, driver)
if RESERVATION != WORNG_MESSAGE:
    logging.info('Reservation Success')
else:
    logging.info('Fail reservation.')

INFO:Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


id:  >تسجيل لأول مرة


INFO:Go Reservation Page.
INFO:Reservation Success


In [69]:
driver = open_driver()
driver.get('https://reg.nid-moi.gov.iq/')
get_data_first_page(person2, driver)
logging.info('Go Reservation Page.')
time.sleep(1)
RESERVATION = get_confirmation_page(person2, driver)
if RESERVATION != WORNG_MESSAGE:
    logging.info('Reservation Success')
else:
    logging.info('Fail reservation.')

INFO:Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1866.237 Safari/537.36


id:  >تسجيل لأول مرة


INFO:Go Reservation Page.
INFO:Reservation Success


In [ ]:
ElementClickInterceptedException, TimeoutException, WebDriverException

In [65]:
driver.find_element_by_xpath('//div[@id="6CfO0QqT5k"]').click()

In [73]:
driver = open_driver()
driver.get('GET', 'https://reg.nid-moi.gov.iq/')

INFO:Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36


TypeError: WebDriver.get() takes 2 positional arguments but 3 were given